In [83]:
import pickle

# Specify the path to your pickle file
pickle_file = "C:\\Users\\omvis\\chess_prediction_project\\train_dataset\\combined_dataset.pkl"

# Load the data
with open(pickle_file, 'rb') as file:
    X, y = pickle.load(file)

In [84]:
len(X)

21456

In [85]:
import pickle

# Specify the path to your pickle file
pickle_file = "C:\\Users\\omvis\\chess_prediction_project\\train_dataset\\features.pkl"

# Load the data
with open(pickle_file, 'rb') as file:
    X_feature = pickle.load(file)

In [86]:
len(X_feature)

21456

In [87]:
X[0]

array([[ 0. ,  0. ,  0. ,  0.6,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. , -0.2, -1. ],
       [ 0. ,  0. ,  0. ,  0. , -0.2,  0. ,  0. , -0.2],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0. ],
       [ 0. ,  0.2,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ],
       [ 0. ,  0. ,  0. ,  0. ,  0. ,  0.2,  0. ,  0.2],
       [ 0. ,  0. ,  0. ,  0. ,  0.5,  0. ,  1. ,  0. ]])

In [88]:
X_feature[0]

array([ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0., 23.,  0.,  0.,
        0., -1.,  2., 14., 14.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [89]:
y[0]

1

In [90]:
import numpy as np
X = np.array(X)
X_feature = np.array(X_feature)
y = np.array(y)

In [91]:
y.shape

(21456,)

In [92]:
from sklearn.preprocessing import MinMaxScaler

# Assuming X_feature is a numpy array of shape (num_samples, 26)
scaler = MinMaxScaler()

# Fit the scaler on X_feature and transform it
X_feature = scaler.fit_transform(X_feature)

In [93]:
X_feature[0]

array([0.        , 0.        , 1.        , 0.475     , 0.55555556,
       0.        , 0.        , 0.        , 0.14285714, 0.5       ,
       0.70503597, 0.5       , 0.        , 0.5       , 0.5       ,
       0.64285714, 0.42424242, 0.42424242, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.34117647,
       0.6       ])

In [94]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

def build_model(board_input_shape=(8, 8, 1), phase_input_shape=(26,)):
    # Input for 8x8 chessboard (single channel)
    board_input = Input(shape=board_input_shape, name='board_input')

    # CNN layers for chessboard (8x8 matrix) with L2 regularization
    x1 = Conv2D(32, (3, 3), activation='relu', padding='same')(board_input)
    x1 = MaxPooling2D((2, 2))(x1)
    
    x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x1)
    x1 = MaxPooling2D((2, 2))(x1)
    Dropout(0.3)
   
    x1 = Flatten()(x1)

    # Input for phase-specific features
    phase_input = Input(shape=phase_input_shape, name='phase_input')

    # Dense layers for phase-specific features with L2 regularization
    x2 = Dense(100, activation='relu', kernel_regularizer=l2(0.1))(phase_input)

    # Concatenate the CNN output and the phase-specific features
    combined = Concatenate()([x1, x2])
    
    # Additional dense layers after concatenation with L2 regularization
    
    x = Dense(100, activation='relu', kernel_regularizer=l2(0.1))(combined)

    # Output layer (3 classes: white, black, equal)
    output = Dense(3, activation='softmax')(x)

    # Create and compile the model
    model = Model(inputs=[board_input, phase_input], outputs=output)
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

model = build_model()

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ board_input         │ (None, 8, 8, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 8, 8, 32)  │        320 │ board_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 4, 4, 32)  │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 4, 4, 64)  │     18,496 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 2, 2, 64)  │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ phase_input         │ (None, 26)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 256)       │          0 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 100)       │      2,700 │ phase_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 356)       │          0 │ flatten_1[0][0],  │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 100)       │     35,700 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 3)         │        303 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 57,519 (224.68 KB)

 Trainable params: 57,519 (224.68 KB)

 Non-trainable params: 0 (0.00 B)

In [95]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets, specifying the test size as 8000
X_train, X_test, X_feature_train, X_feature_test, y_train, y_test = train_test_split(
    X, X_feature, y, test_size=4000, random_state=42
)

# Verify the split
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")



Training set size: 17456 samples
Test set size: 4000 samples


In [96]:
from sklearn.model_selection import KFold
import numpy as np

# Assuming X_train, X_feature_train, y_train are numpy arrays
# Here, we will apply KFold to all datasets: X_train, X_feature_train, and y_train

kfold = KFold(n_splits=6, shuffle=True, random_state=42)

for train_idx, val_idx in kfold.split(X_train, y_train):
    # Split the data into training and validation sets
    X_train_fold, X_val_fold = X_train[train_idx], X_train[val_idx]
    X_feature_train_fold, X_feature_val_fold = X_feature_train[train_idx], X_feature_train[val_idx]
    y_train_fold, y_val_fold = y_train[train_idx], y_train[val_idx]

    # Train the model using the training fold and validate using the validation fold
    model_trained = model.fit(
        [X_train_fold, X_feature_train_fold], y_train_fold,  # Train with both chessboard and phase features
        epochs=10,
        validation_data=([X_val_fold, X_feature_val_fold], y_val_fold)  # Validate with both chessboard and phase features
    )


Epoch 1/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.4519 - loss: 6.2715 - val_accuracy: 0.4863 - val_loss: 1.0454
Epoch 2/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5184 - loss: 1.0279 - val_accuracy: 0.5509 - val_loss: 0.9919
Epoch 3/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5652 - loss: 0.9730 - val_accuracy: 0.5038 - val_loss: 1.0169
Epoch 4/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5786 - loss: 0.9463 - val_accuracy: 0.5900 - val_loss: 0.9227
Epoch 5/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5994 - loss: 0.9158 - val_accuracy: 0.5966 - val_loss: 0.9064
Epoch 6/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6122 - loss: 0.8962 - val_accuracy: 0.5938 - val_loss: 0.9180
Epoch 7/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6051 - loss: 0.9027 - val_accuracy: 0.5880 - val_loss: 0.9195
Epoch 8/10
455/455 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6117 - loss: 0.8862 - val_accuracy: 0.

In [97]:
model.evaluate(
    [X_test,X_feature_test],  # Provide the test data (both board and phase-specific features)
    y_test                  # Provide the true labels
)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6174 - loss: 0.8743


[0.8800085783004761, 0.6177499890327454]

In [114]:
import pickle

# Specify the path to your pickle file
pickle_file = "C:\\Users\\omvis\\chess_prediction_project\\test_dataset\\test_dataset.pkl"

# Load the data
with open(pickle_file, 'rb') as file:
    Xtest, id = pickle.load(file)

In [115]:
import pickle

# Specify the path to your pickle file
pickle_file = "C:\\Users\\omvis\\chess_prediction_project\\test_dataset\\features_test.pkl"

# Load the data
with open(pickle_file, 'rb') as file:
    Xtest_feature = pickle.load(file)

In [116]:
Xtest = np.array(Xtest)
Xtest_feature = np.array(Xtest_feature)
id=np.array(id)

In [117]:
y_predicted = model.predict([Xtest,Xtest_feature])
y_predicted[0]

267/267 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


array([0.49849933, 0.3759637 , 0.12553702], dtype=float32)

In [118]:
y_predicted = np.argmax(y_predicted,axis=1)

In [119]:
y_predicted[:5]

array([0, 0, 0, 2, 1], dtype=int64)

In [120]:
len(y_predicted)

8539

In [121]:
import pandas as pd
df = pd.read_csv("C:\\Users\\omvis\\chess_prediction_project\\chess_test_csv.csv")
df.shape

(8540, 2)

In [122]:
import warnings
warnings.filterwarnings("ignore")
for idx, val in enumerate(df.image_id):
    index = np.where(id == val)[0]  # [0] gets the array of indices
    if y_predicted[index] == 0:
        df['evaluation'][idx] = "equal"
    elif y_predicted[index] == 1:
        df['evaluation'][idx] = "white"
    else:
        df['evaluation'][idx] = "black"

In [123]:
df.rename(columns={'image_id' : 'id' },inplace=True)

In [124]:
df.head(3)

,id,evaluation
0,26065374,equal
1,32481096,black
2,36997677,white


In [125]:
df = df[~df['id'].duplicated(keep='first')]

In [126]:
len(df['id'].unique())

8539

In [111]:
df.nunique

<bound method DataFrame.nunique of             id evaluation
0     26065374      equal
1     32481096      black
2     36997677      white
3     34417938      equal
4     28905181      black
...        ...        ...
8535  31917698      white
8536    393246      equal
8537  26319610      black
8538  16378115      black
8539  30711130      white

[8539 rows x 2 columns]>

In [112]:
df['evaluation'].isna().sum()

0

In [113]:
df.to_csv("result.csv",index=False)